In [31]:
from rich import print
import os
import pathlib
import random
import sys
from asyncio.log import logger
from collections import defaultdict
from dataclasses import dataclass
from math import floor
from typing import Any, Dict, List, Optional, Tuple, Union

from torchtyping import TensorType

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import torch
import torch.nn as nn
import tqdm
from hydra_zen import builds, instantiate
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, RandomCrop, Resize, ToTensor
from traitlets import Int
from capit.core.data.config import ImageShape, ModalityConfig
from capit.core.utils.storage import load_json, save_json

from capit.decorators import configurable
from capit.utils import get_logger
from torch.utils.data.dataloader import default_collate

logger = get_logger(__name__)

os.environ["HYDRA_FULL_ERROR"] = "1"

In [32]:
%load_ext autoreload
%autoreload 2
from capit.core.data.datasets import (
    get_image_transforms_instait,
    InstagramImageTextMultiModalDatasePyArrow,
    ChallengeSamplesSourceTypes,
    SplitType,
)

root_filepath = pathlib.Path("/data/")
dataset_config = InstagramImageTextMultiModalDatasePyArrow.build_config(
    populate_full_signature=True
)
dataset_config_instance = dataset_config(dataset_dir=root_filepath)
train_dataset = InstagramImageTextMultiModalDatasePyArrow(
    dataset_dir=root_filepath,
    top_k_percent=25,
    image_transforms=get_image_transforms_instait(),
    max_num_collection_images_per_episode=0,
    max_num_query_images_per_episode=25,
    challenge_image_source=ChallengeSamplesSourceTypes.WITHIN_USER,
    num_episodes=1000,
    set_name=SplitType.TRAIN,
)

val_dataset = InstagramImageTextMultiModalDatasePyArrow(
    dataset_dir=root_filepath,
    top_k_percent=25,
    image_transforms=get_image_transforms_instait(),
    max_num_collection_images_per_episode=0,
    max_num_query_images_per_episode=25,
    challenge_image_source=ChallengeSamplesSourceTypes.WITHIN_USER,
    num_episodes=1000,
    set_name=SplitType.VAL,
)

test_dataset = InstagramImageTextMultiModalDatasePyArrow(
    dataset_dir=root_filepath,
    top_k_percent=25,
    image_transforms=get_image_transforms_instait(),
    max_num_collection_images_per_episode=0,
    max_num_query_images_per_episode=25,
    challenge_image_source=ChallengeSamplesSourceTypes.WITHIN_USER,
    num_episodes=1000,
    set_name=SplitType.TEST,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import tqdm

train_usernames = train_dataset.set_usernames
val_usernames = val_dataset.set_usernames
test_usernames = test_dataset.set_usernames
with tqdm.tqdm(total=len(train_usernames)) as pbar:
    for username in train_usernames:
        assert username not in val_usernames
        assert username not in test_usernames
        pbar.update(1)

with tqdm.tqdm(total=len(val_usernames)) as pbar:
    for username in val_usernames:
        assert username not in train_usernames
        assert username not in test_usernames
        pbar.update(1)

with tqdm.tqdm(total=len(test_usernames)) as pbar:
    for username in test_usernames:
        assert username not in train_usernames
        assert username not in val_usernames
        pbar.update(1)

100%|██████████| 3065/3065 [00:01<00:00, 2749.93it/s]


In [41]:
train_dataset = InstagramImageTextMultiModalDatasePyArrow(
    dataset_dir=root_filepath,
    top_k_percent=25,
    image_transforms=get_image_transforms_instait(),
    max_num_collection_images_per_episode=0,
    max_num_query_images_per_episode=25,
    challenge_image_source=ChallengeSamplesSourceTypes.WITHIN_USER,
    num_episodes=1000,
    set_name=SplitType.TRAIN,
)
for idx, item in enumerate(train_dataset):
    if idx == 100:
        break

47
6
64
69
73
53
55
70
70
62
57


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_20406/3104084952.py:11 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_20406/3104084952.py'                        │
│                                                                                                  │
│ /codebase/capit/core/data/datasets.py:796 in __getitem__                                         │
│                                                                                                  │
│    793 │   │   │   return self.__getitem__(index + 1)                                            │
│    794 │   │                                                                                     │
│    795 │   │   for image_path, info_path in challenge_posts:                                     │
│ ❱  796 │   │   │   image, text = self.read_image_caption(image_path, info_path)                  │
│    797 │   │   │   data_dict["challenge_images"].append(image)                                   │
│    798 │   │   │   data_dict["challenge_paths"].append(dict(image=image_path, info=info_path))   │
│    799                                                                                           │
│                                                                                                  │
│ /codebase/capit/core/data/datasets.py:686 in read_image_caption                                  │
│                                                                                                  │
│    683 │   │                                                                                     │
│    684 │   │   image = Image.open(image_path)                                                    │
│    685 │   │   if self.image_transforms is not None:                                             │
│ ❱  686 │   │   │   image = self.image_transforms(image)                                          │
│    687 │   │                                                                                     │
│    688 │   │   if self.text_transforms is not None:                                              │
│    689 │   │   │   text = self.text_transforms(text)                                             │
│                                                                                                  │
│ /opt/conda/envs/main/lib/python3.10/site-packages/torchvision/transforms/transforms.py:95 in     │
│ __call__                                                                                         │
│                                                                                                  │
│     92 │                                                                                         │
│     93 │   def __call__(self, img):                                                              │
│     94 │   │   for t in self.transforms:                                                         │
│ ❱   95 │   │   │   img = t(img)                                                                  │
│     96 │   │   return img                                                                        │
│     97 │                                                                                         │
│     98 │   def __repr__(self) -> str:                                                            │
│                                                                                                  │
│ /opt/conda/envs/main/lib/python3.10/site-packages/torch/nn/modules/module.py:1194 in _call_impl  │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forwar

In [27]:
%load_ext autoreload
%autoreload 2
import capit

print(capit)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'capit' (<_frozen_importlib_external._NamespaceLoader object at 0x7fa3a8087760>)>